# Outil de cartographie: carte de chaleur (France) par département



Il est nécessaire de télécharger les limites géographiques des départements de France disponibles sur le Github: c'est le dossier "france_departements". Il faut ensuite donner le chemin vers le fichier "France_departements-polygons.shp" contenu dans ce dossier.

In [ ]:

#@title Installation des bibliothèques nécessaires et connexion à un compte Google Drive.

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''


## Installation des bibliothèques et connexion au compte Google Drive


!pip install pgeocode==0.3.0
!pip install geopandas==0.9.0
from google.colab import drive
import json
import pgeocode
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
%matplotlib inline

drive.mount('/content/drive/')
%cd /content/drive/My Drive/romans_scolaires/regions_map/

In [ ]:
#@markdown Le fichier d'entrée des données est un fichier .JSON contenant les lieux ainsi que les départements auxquels ils se rapportent
chemin_fichier_json = "/content/drive/My Drive/romans_scolaires/test_textblocks.json" #@param {type:"string"}

liste_codes = []

with open(chemin_fichier_json, "r") as f :
  contenu_json = json.load(f)
  for lieu in contenu_json:
    code = contenu_json[lieu]["Code Postal"]
    liste_codes.append(code)

nomi = pgeocode.Nominatim('fr')
df_tout = nomi.query_postal_code(liste_codes)
df_tout.to_excel('df_tout.xlsx', index = True, header=True)

## Compter nombre par département
df_nbr_depart=df_tout['county_name'].value_counts()
df_nbr_depart.to_excel('df_nbr_depart.xlsx', index = True, header=True)
df2_nbr_depart = pd.DataFrame(df_nbr_depart)



#@markdown Chemin vers le fichier "France_departements-polygons.shp":
chemin_fichier_shp = '/content/drive/My Drive/romans_scolaires/regions_map/mygeodata/France_departements-polygon.shp' #@param {type:"string"}

departs = gpd.read_file(chemin_fichier_shp)
departs.to_excel('voir_departs.xlsx', index = True, header=True)
departs.plot(figsize=(20,10))
merged = departs.set_index('nom').join(df2.set_index('nom'))
merged = merged.reset_index()
merged = merged.fillna(0)
merged[['nom', 'geometry', 'county_name', 'code']]
fig, ax = plt.subplots(1, figsize=(20, 10))
ax.axis('off')
color = 'Reds'
vmin, vmax = 4, 93
sm = plt.cm.ScalarMappable(cmap=color, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
cbar.ax.tick_params(labelsize=15)
cbar.ax.set_title('Nombre de lieux cités par départements \n sur l\'ensemble du corpus de romans scolaires', fontdict={'fontsize': '13', 'fontweight' : '1'})
merged.plot('county_name', cmap=color, linewidth=0.8, ax=ax, edgecolor='0.8', figsize=(20,10))

#@markdown Nom du fichier avec la carte (sortie) au format .png:
nom_carte_png = 'carte_departements.png' #@param {type:"string"}


plt.savefig(nom_carte_png)